# Класс для решения матричных игр

Ниже, на языке Python, описан класс `MatrixGame`, который умеет решать матричные игры и кое-что ещё. 
Большинство приведённых примеров, описанных методов и математические обозначения, используемые здесь, взяты из учебного пособия Васина А.А. и Морозова В.В. "Введение в теорию игр с приложениями к экономике". Код написан мною 😎.

In [1]:
"""
Created on Fri Oct 30 2020

@author: Evgeny Laba
"""

import numpy as np
import pandas as pd

class MatrixGame:
    def __init__(self, X: np.ndarray):
        self.__P = X
        
    def PayoffMatrix(self):
        return self.__P
        
    def minimax(self):
        return np.apply_along_axis(np.max, 0, self.__P).min()
    
    def maximin(self):
        return np.apply_along_axis(np.min, 1, self.__P).max()
    
    def saddle_points(self):
        n = self.__P.shape[0]
        m = self.__P.shape[1]
        S = np.zeros((n,m), dtype = int)
        for i in range(n):
            for j in range(m):
                if((self.__P[i][j] <= self.__P[i]).all() and (self.__P[i][j] >= self.__P[:,j]).all()):
                    S[i][j] = 1
        return S
    
    def Browns_method(self, eps = 1/1000, max_iter = 10000, iter_data = False):
        
        # initialization
        i = np.random.randint(0, self.__P.shape[0], 1)
        j = np.random.randint(0, self.__P.shape[1], 1)
        k = t1 = t2 = 1
        c = self.__P.T[j[-1]]
        r = self.__P[i[-1]]
        v1 = v1b = c.max()
        v2 = v2b = r.min()
        if(iter_data == True):
            df = np.concatenate((np.array([i[-1] + 1]), 
                                 c, 
                                 np.array([v1, j[-1] + 1]), 
                                 r, 
                                 np.array([v2, v1b, v2b, v1b - v2b]))) 
        
        # iteration and updates
        while((v1b - v2b > eps) and (max_iter > k)):
            i = np.append(i, np.random.choice((c == c.max()).nonzero()[0], 1)[0])
            j = np.append(j, np.random.choice((r == r.min()).nonzero()[0], 1)[0])
            k += 1
            c = c + self.__P.T[j[-1]]
            r = r + self.__P[i[-1]]
            v1 = c.max()/k
            v2 = r.min()/k
            if(v1 < v1b):
                v1b = v1
                t1 = k
            if(v2 > v2b):
                v2b = v2
                t2 = k
            if(iter_data == True):
                df = np.vstack((df, np.concatenate((np.array([i[-1] + 1]), 
                                c, 
                                np.array([v1, j[-1] + 1]), 
                                r, 
                                np.array([v2, v1b, v2b, v1b - v2b]))) ))
                
        # dataframe wrap
        if(iter_data == True):
            l = c.shape[0]
            m = r.shape[0]
            df = pd.DataFrame(df)
            cnames = ["i"]
            df.rename(columns = {df.columns[n]: cnames[n] for n in range(1)}, inplace = True)
            cnames = []
            for n in range(l):
                cnames.append(f"c{n + 1}")
            df.rename(columns = {df.columns[n]: cnames[n - 1] for n in range(1, 
                                                                             1 + l)}, inplace = True)
            if(np.issubdtype(self.__P[0,0], int) == True):
                for n in cnames:
                     df[n] = np.int64(df[n])
            cnames = ["v1", "j"]
            df.rename(columns = {df.columns[n]: cnames[n - 1 - l] for n in range(1 + l, 
                                                                                 3 + l)}, inplace = True)
            cnames = []
            for n in range(m):
                cnames.append(f"r{n + 1}")
            df.rename(columns = {df.columns[n]: cnames[n - 3 - l] for n in range(3 + l, 
                                                                                 3 + l + m)}, inplace = True)
            if(np.issubdtype(self.__P[0,0], int) == True):
                for n in cnames:
                     df[n] = np.int64(df[n])
            cnames = ["v2", "v1*", "v2*", "v1*-v2*"]
            df.rename(columns = {df.columns[n]: cnames[n - 3 - l - m] for n in range(3 + l + m,
                                                                                     7 + l + m)}, inplace = True)
            df[["i", "j"]] = df[["i", "j"]].apply(np.int64)
            df.index = range(1, len(df) + 1)
        
        # output if method converges 
        if(v1b - v2b < eps):
            p = np.zeros(r.shape[0])
            q = np.zeros(c.shape[0])
            for n in range(p.shape[0]):
                p[n] = ((i[0:t2] == n) * 1).sum()/t2
            for n in range(q.shape[0]):
                q[n] = ((j[0:t1] == n) * 1).sum()/t1
            if(iter_data == True):
                return [p, q, (v1b + v2b)/2, df]
            else:
                return [p, q, (v1b + v2b)/2]
        # or doesn't converge
        else:
            print(f"Algorithm hasn't converged in {max_iter} iteration steps!")
            if(iter_data == True):
                return df
            else:
                pass
            
    def solve(self):
        # if solution exists in pure strategies
        if(self.minimax() == self.maximin()):
            print("Solution in pure strategies is found!")
            return [np.unique(np.where(self.saddle_points() == 1)[0] + 1),
                    np.unique(np.where(self.saddle_points() == 1)[1] + 1),
                    self.minimax()]
        # else searching mixed strategies solution using Brown's method
        else:
            G = self.Browns_method()
            # if method has converged
            if(G == None):
                print("Therefore no solution is found!")
            # otherwise
            else:
                print("Solution in mixed strategies is found!")
                return G

Опишем подробнее, что умеет класс. На вход данный класс всегда принимает платёжную матрицу в виде `numpy.ndarray`. На основе введённой матрицы можно сделать следуюшие вещи:
* посчитать минимакс $\overline{v}$
* посчитать максимин $\underline{v}$
* найти все седловые точки
* решить матричную игру либо в чистых стратегиях, либо попробовать решить её в смешанных стратегиях при помощи метода Брауна.

В качестве первого примера рассмотрим игру со следующей платёжной матрицей


$$
P = \begin{bmatrix}
     7 & -1 & -4 &  1\\
     4 &  2 &  3 &  2\\
     2 &  2 &  5 &  2\\
     4 & -3 &  7 & -2
\end{bmatrix}
$$

Здесь игра имеет решение в чистых стратегиях и $ \underline{v} = \overline{v} = 2, X^0 = \{2,3\}, Y^0 = \{2,4\} $. Четыре седловые точки образуют множество $ X^0 \times Y^0 $. Теперь найдём указанные значения при помощи описанного класса.


In [2]:
P = np.array([[7,-1,-4, 1],
              [4, 2, 3, 2],
              [2, 2, 5, 2],
              [4,-3, 7,-2]])

G = MatrixGame(P)

In [3]:
G.minimax()

2

In [4]:
G.maximin()

2

In [5]:
G.saddle_points()

array([[0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 0, 0]])

Ну и, собственно, решим саму игру используя метод `solve`. В случае решения в чистых стратегиях метод возвращает список $ \{ X^0, Y^0, v \}$.

In [6]:
G.solve()

Solution in pure strategies is found!


[array([2, 3]), array([2, 4]), 2]

Тут всё достаточно очевидно. 

Рассмотрим теперь случай, когда решения в чистых стратегиях нет. Пусть дана следующая платёжная матрица

$$
P = \begin{bmatrix}
     2 &  1 &  0 \\
     2 &  0 &  3 \\
    -1 &  3 & -3 
\end{bmatrix}.
$$

Здесь $ \underline{v} = 0 \ne \overline{v} = 2 \implies $ решения в чистых стратегиях нет. При этом существует решение игры в смешанных стратегиях, которое имеет вид

$$
p^0 = q^0 = (0, 2/3, 1/3), v = 1.
$$

Класс ищет приблизительное решение с заданной погрешностью при помощи метода Брауна подробно описанного в пособии, которое указано в начале.  В случае нахождения решения метод возвращает список $ \{ p(t_2), q(t_1), v^* = \frac{v_1^* +v_2^*}{2} \}$. Давайте решим игру при помощи `solve`. Так как в методе используется генерация случайнных данных, для повторения результов, не забываем инициализировать `numpy.random.seed` одинаковыми значениями.

In [7]:
P = np.array([[ 2, 1, 0], 
              [ 2, 0, 3],
              [-1, 3,-3]])

G = MatrixGame(P)

np.random.seed(73)

G.solve()

Solution in mixed strategies is found!


[array([0.        , 0.66666667, 0.33333333]),
 array([0.        , 0.66666667, 0.33333333]),
 1.0]

Вуаля! Получили идеальный результат 😎.

При поиске решения в смешанных стратегиях методом `solve`, в методе Брауна всегда используется параметр `eps`, по умолчанию равный 1/1000. Также фиксирован параметр `max_iter`, который контролирует число шагов, за которое алгоритм должен успеть сойтись. Для настраиваемого метода Брауна существует основной метод, называется который, как не удивительно - `Browns_method`. В нём можно менять вышеуказанные параметры. В дополнение там есть дополнительный логический параметр `iter_data`, поменяв значение, которого на `True`, позволяет в списке с решением возвращать таблицу с пошаговой историей итераций. В принципе, это особо не нужно, разве только для наглядности или проверки правильности работы метода. Вот предыдущий пример с таблицей.

In [8]:
np.random.seed(73)

G.Browns_method(iter_data = True)

[array([0.        , 0.66666667, 0.33333333]),
 array([0.        , 0.66666667, 0.33333333]),
 1.0,
     i  c1  c2  c3        v1  j  r1  r2  r3        v2       v1*  v2*   v1*-v2*
 1   3   0   3  -3  3.000000  3  -1   3  -3 -3.000000  3.000000 -3.0  6.000000
 2   2   0   6  -6  3.000000  3   1   3   0  0.000000  3.000000  0.0  3.000000
 3   2   0   9  -9  3.000000  3   3   3   3  1.000000  3.000000  1.0  2.000000
 4   2   0  12 -12  3.000000  3   5   3   6  0.750000  3.000000  1.0  2.000000
 5   2   1  12  -9  2.400000  2   7   3   9  0.600000  2.400000  1.0  1.400000
 6   2   2  12  -6  2.000000  2   9   3  12  0.500000  2.000000  1.0  1.000000
 7   2   3  12  -3  1.714286  2  11   3  15  0.428571  1.714286  1.0  0.714286
 8   2   4  12   0  1.500000  2  13   3  18  0.375000  1.500000  1.0  0.500000
 9   2   5  12   3  1.333333  2  15   3  21  0.333333  1.333333  1.0  0.333333
 10  2   6  12   6  1.200000  2  17   3  24  0.300000  1.200000  1.0  0.200000
 11  2   7  12   9  1.090909  2  

Видно, что алгоритм сошёлся за 12 шагов. Разница между $ v^*_1 $ и $ v^*_2 $ равна нулю, что удовлетворяет любому сколь угодно малому $\epsilon$.

Однако если мы запустим метод повторно - результат будет несколько хуже.

In [9]:
G.Browns_method(iter_data = True)

[array([0.00099108, 0.66600595, 0.33300297]),
 array([0.07142857, 0.64285714, 0.28571429]),
 0.9995044598612488,
       i   c1   c2    c3        v1  j    r1    r2    r3        v2  v1*  \
 1     3    2    2    -1  2.000000  1    -1     3    -3 -3.000000  2.0   
 2     1    2    5    -4  2.500000  3     1     4    -3 -1.500000  2.0   
 3     2    2    8    -7  2.666667  3     3     4     0  0.000000  2.0   
 4     2    2   11   -10  2.750000  3     5     4     3  0.750000  2.0   
 5     2    2   14   -13  2.800000  3     7     4     6  0.800000  2.0   
 ...  ..  ...  ...   ...       ... ..   ...   ...   ...       ...  ...   
 1005  3  685  965  1085  1.079602  2  1014   997  1020  0.992040  1.0   
 1006  3  686  965  1088  1.081511  2  1013  1000  1017  0.994036  1.0   
 1007  3  687  965  1091  1.083416  2  1012  1003  1014  0.996028  1.0   
 1008  3  688  965  1094  1.085317  2  1011  1006  1011  0.998016  1.0   
 1009  3  689  965  1097  1.087215  2  1010  1009  1008  0.999009  1.0   

В этот раз потребовалось более тысячи итераций. И полученные значения уже не такие 'красивые', хотя и удовлетворяют заданным требованиям. 

В заключение запустим метод ещё несколько раз на матрице из первого примера. Напомню, что эта игра имеет решение в чистых стратегиях.

In [10]:
P = np.array([[7,-1,-4, 1],
              [4, 2, 3, 2],
              [2, 2, 5, 2],
              [4,-3, 7,-2]])

G = MatrixGame(P)

G.Browns_method(iter_data = True)

[array([0., 1., 0., 0.]),
 array([0.00000000e+00, 9.99333333e-01, 3.33333333e-04, 3.33333333e-04]),
 2.0004999999999997,
       i    c1    c2    c3    c4        v1  j    r1    r2     r3    r4  \
 1     2    -4     3     5     7  7.000000  3     4     2      3     2   
 2     4    -3     5     7     5  3.500000  4     8    -1     10     0   
 3     3    -4     7     9     2  3.000000  2    10     1     15     2   
 4     3    -5     9    11    -1  2.750000  2    12     3     20     4   
 5     3    -6    11    13    -4  2.600000  2    14     5     25     6   
 ...  ..   ...   ...   ...   ...       ... ..   ...   ...    ...   ...   
 2996  3 -2997  5993  5995 -8977  2.001001  2  5996  5987  14980  5988   
 2997  3 -2998  5995  5997 -8980  2.001001  2  5998  5989  14985  5990   
 2998  3 -2999  5997  5999 -8983  2.001001  2  6000  5991  14990  5992   
 2999  3 -3000  5999  6001 -8986  2.001000  2  6002  5993  14995  5994   
 3000  3 -3001  6001  6003 -8989  2.001000  2  6004  5995  15000 

In [11]:
G.Browns_method(iter_data = True)

[array([0., 0., 1., 0.]),
 array([3.33333333e-04, 9.99333333e-01, 3.33333333e-04, 0.00000000e+00]),
 2.0004999999999997,
       i    c1    c2    c3    c4        v1  j    r1    r2     r3    r4  \
 1     3    -4     3     5     7  7.000000  3     2     2      5     2   
 2     4     3     7     7    11  5.500000  1     6    -1     12     0   
 3     4     2     9     9     8  3.000000  2    10    -4     19    -2   
 4     2     1    11    11     5  2.750000  2    14    -2     22     0   
 5     3     0    13    13     2  2.600000  2    16     0     27     2   
 ...  ..   ...   ...   ...   ...       ... ..   ...   ...    ...   ...   
 2996  2 -2991  5995  5995 -8971  2.001001  2  8994  5982  11986  5984   
 2997  3 -2992  5997  5997 -8974  2.001001  2  8996  5984  11991  5986   
 2998  2 -2993  5999  5999 -8977  2.001001  2  9000  5986  11994  5988   
 2999  2 -2994  6001  6001 -8980  2.001000  2  9004  5988  11997  5990   
 3000  3 -2995  6003  6003 -8983  2.001000  2  9006  5990  12002 

In [12]:
G.Browns_method(iter_data = True)

Algorithm hasn't converged in 10000 iteration steps!


,i,c1,c2,c3,c4,v1,j,r1,r2,r3,r4,v2,v1*,v2*,v1*-v2*
1,1,7,4,2,4,7.0000,1,7,-1,-4,1,-4.000000,7.0000,-4.000000,11.000000
2,1,3,7,7,11,5.5000,3,14,-2,-8,2,-4.000000,5.5000,-4.000000,9.500000
3,4,-1,10,12,18,6.0000,3,18,-5,-1,0,-1.666667,5.5000,-1.666667,7.166667
4,4,-2,12,14,15,3.7500,2,22,-8,6,-2,-2.000000,3.7500,-1.666667,5.416667
5,4,-3,14,16,12,3.2000,2,26,-11,13,-4,-2.200000,3.2000,-1.666667,4.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,3,-9994,19996,19998,-29961,2.0006,2,20008,19971,49968,19978,1.997899,2.0006,1.997899,0.002701
9997,3,-9995,19998,20000,-29964,2.0006,2,20010,19973,49973,19980,1.997899,2.0006,1.997899,0.002701
9998,3,-9996,20000,20002,-29967,2.0006,2,20012,19975,49978,19982,1.997900,2.0006,1.997900,0.002701
9999,3,-9997,20002,20004,-29970,2.0006,2,20014,19977,49983,19984,1.997900,2.0006,1.997900,0.002700


Как видим, в этих случаях метод тоже работает, правда, несколько хуже. В последней попытке метод и вовсе не сошёлся при заданных параметрах...

PS. TO DO. Что можно сделать ещё? Например
* реализовать доминирование 
* находить решение игры методом Гаусса.

Но это в уже в следующий раз.

PSS. Если нашли в тексте или коде какие-либо ошибки - сообщите мне 😉.